In [19]:
pwd

'/Users/surajpawar/Desktop/Keras_NLP'

In [1]:
!pip install -q --upgrade keras-nlp

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras_core as keras

Using JAX backend.


In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9995k      0  0:00:08  0:00:08 --:--:-- 14.1M


In [5]:
BATCH_SIZE = 16

imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size = BATCH_SIZE

)

imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size = BATCH_SIZE

)

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [6]:
print(imdb_train.unbatch().take(1).get_single_element())

(<tf.Tensor: shape=(), dtype=string, numpy=b"In the small American town of Meadowvale Dr. Anthony Blake (David Gale, the IMDb listing for this character is wrong it's definitely Dr. Blake not Dr. Blakely) is the director and founder of the famed 'Psychological Research Institute' and also host's a local T.V. programme called 'Independent Thinkers'. He uses this T.V. show to hypnotise the viewers and make them commit acts of violence. Dr. Blake has the help of a large brain with an evil face that uses it's spinal cord as a tail thingy. Usually the brain is just sitting in a tank, eats mice and the odd bad actor, each time it eats someone it gets much bigger. Meanwhile at the local high-school gifted but troublesome teenager Jim Majelewski (Tom Bresnahan) has been caught putting Sodium down the toilets. Jim is sent to Dr. Blake at the PRI for help with his attitude and behavioural problems. While there Dr. Blake hooks Jim up to, well something I'm not actually sure what. This whatever it

In [26]:
pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.6 MB/s eta 0:00:0000:0100:01
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)


  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.0
    Uninstalling tensorflow-2.12.0:
      Successfully uninstalled tensorflow-2.12.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])

17596448/17596448 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[-1.5387032,  1.5418267]], dtype=float32)

In [8]:
classifier.evaluate(imdb_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 317s 202ms/step - loss: 0.4538 - sparse_categorical_accuracy: 0.7905


[0.46284598112106323, 0.783519983291626]

In [9]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes = 2

)



231508/231508 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step     
17602216/17602216 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [11]:
classifier.fit(
    imdb_train,
    validation_data = imdb_test,
    epochs = 1

)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1737s 1s/step - loss: 0.5186 - sparse_categorical_accuracy: 0.7323 - val_loss: 0.3106 - val_sparse_categorical_accuracy: 0.8680


In [14]:
import tensorflow as tf

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)


imdb_train_cached = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_cached = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", preprocessor=None, num_classes=2
)
classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3,
)



Epoch 1/3


2023-08-07 21:51:42.339086: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1655s 1s/step - loss: 0.5185 - sparse_categorical_accuracy: 0.7321 - val_loss: 0.3393 - val_sparse_categorical_accuracy: 0.8530
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1465s 937ms/step - loss: 0.2883 - sparse_categorical_accuracy: 0.8808 - val_loss: 0.2963 - val_sparse_categorical_accuracy: 0.8775
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1442s 923ms/step - loss: 0.2163 - sparse_categorical_accuracy: 0.9172 - val_loss: 0.3256 - val_sparse_categorical_accuracy: 0.8732
